<a href="https://colab.research.google.com/github/rakkhrok/bsi_method_analysis/blob/main/gtsrb_architecture_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Drive to access the data
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ls: cannot access '/content/drive/GoogleCollab': No such file or directory


In [ ]:
# Check that the drive is mounted
!ls "/content/drive/My Drive/GoogleCollab"

gtsrb_architecture_training.ipynb  gtsrb_dataset


In [ ]:
# Reference https://www.kaggle.com/code/kar0n7/traffic-signal-recognizer-98-accuracy
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0
